In [ ]:
import search
from sudoku import Sudoku
import copy
import random

In [ ]:
with open('100sudoku.txt', 'r') as f:
    line = f.readline()

In [ ]:
class LewisSudokuProblem(search.Problem):
    
    def __init__(self, initial_sudoku, goal=None):
        self.goal = None
        
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                numbers = set(range(1, 10)) - set(initial_sudoku.get_square(i, j))
                for x in range(i, i + 3):
                    for y in range(j, j + 3):
                        if initial_sudoku.sudoku[x][y] == 0:
                            n = random.choice(list(numbers))
                            initial_sudoku.sudoku[x][y] = n
                            numbers -= set([n])
        self.initial = initial_sudoku
        
    def actions(self, state):
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                square_non_fixed = []
                for x in range(i, i + 3):
                    for y in range(j, j + 3):
                        if (x, y) not in state.fixed:
                            square_non_fixed.append((x, y))
                while len(square_non_fixed) > 1:
                    s1 = square_non_fixed.pop()
                    for s2 in square_non_fixed:
                        yield (s1, s2)
                        
    def result(self, state, action):
        (x1, y1), (x2, y2) = action
        state_copy = copy.deepcopy(state)
        state_copy.sudoku[x1][y1], state_copy.sudoku[x2][y2] = state_copy.sudoku[x2][y2], state_copy.sudoku[x1][y1]
        return state_copy
    
    def goal_test(self, state):
        return self.value(state) == 0
    
    def value(self, state):
        one_to_nine = set(range(1, 10))
        s = 0
        for i in range(0, 9):
            s += len(one_to_nine - set(state.get_line(i)))
            s += len(one_to_nine - set(state.get_column(i)))
        return -s

In [ ]:
def sim(t):
    p = 0.8*0.99**t
    if p < 0.01:
        return 0
    return p

In [ ]:
def tests(lines, problem_class, search_f):
    count = []
    cost = []
    for l in lines:
        s = Sudoku(l)
        p = problem_class(s)
        sudo, c = search_f(p)
        count.append(c)
        cost.append(p.value(sudo))
    return cost, count

In [ ]:
with open('100sudoku.txt', 'r') as f:
    lines = f.readlines()

In [ ]:
hill_cost, hill_count = tests(lines, LewisSudokuProblem, search.hill_climbing)
annealing_cost, annealing_count = tests(lines[:3], LewisSudokuProblem, lambda p: search.simulated_annealing(p, sim))